In [ ]:
#Mounting The Google Drive

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Importing the libraries into the file.
import numpy as np 
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from mlxtend.evaluate import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
from matplotlib.pyplot import subplots
from keras.callbacks import *

In [ ]:
#Reading the csv file which has stored the pixel value of all the images of dataset to be used
dataset = pd.read_csv("/content/drive/MyDrive/A_Z Handwritten Data.csv").astype('float32')
dataset.rename(columns={'0':'label'}, inplace=True)

#Pixel values of Images 
x= dataset.drop('label',axis = 1)

#Seperating the column which has labels of all the data items
y = dataset['label']

In [ ]:
# We are spliting the dataset into train,test and validation dataset.

x_train,x_test,y_train,y_test = train_test_split(x,y)

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3)
x_train,inputs_validation,y_train,targets_validation=train_test_split(x_train,y_train,test_size=0.2)

#Scaling the values of the pixels in the different dataset. 
standard_scaler = MinMaxScaler()
standard_scaler.fit(x_train)

x_train = standard_scaler.transform(x_train)
x_test = standard_scaler.transform(x_test)

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [ ]:
#Defining the Architecture Of the ANN Model to be used. 

model = keras.Sequential()
model.add(layers.Dense(100, activation="relu" , input_dim = x_train.shape[1]))
model.add(layers.Dense(len(y.unique()), activation="softmax"))
adam = keras.optimizers.Adam(learning_rate=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=adam , metrics = ['Accuracy'])

In [ ]:
#Training the model with the training dataset

model.fit(x_train,y_train,epochs=5)

In [ ]:
#Evaluating the Model on Test Dataset
model.evaluate(x_test,y_test)

In [ ]:
#Storing the predicted values of data items in test dataset.
y_pred=model.predict(x_test)

2910/2910 [==============================] - 5s 2ms/step


In [ ]:
#Function To get appropriate Alphabet based on the predictions of the Model
#The model returns an array of 26 size describing the probabilities of each character.
#We take the character with max probability as the predicted result for that sample image.

def get_alphabet(y):
  s="ABCDEFGHIJKLMNOPQRSTUVWXYZ"
  val=[0]*len(y)
  for j in range (len(y)):
   val[j]=s[np.argmax(y[j])]
  return val

In [ ]:
#Saving the model in the drive to later extract weight from it.

model.save_weights('gfgModelWeights.h5')
print('Model Saved!')

In [ ]:
#Calling the get_alphabet() to get the respective value of alphabets.
y_test=get_alphabet(y_test)
y_pred=get_alphabet(y_pred)

In [ ]:
#Displaying the confusion matrix based upon the values of Predicted value and actual Values Of Test Dataset

disp = metrics.ConfusionMatrixDisplay.from_predictions(y_test, y_pred)
fig = disp.ax_.get_figure() 
fig.set_figwidth(10)
fig.set_figheight(10)  

In [ ]:
loc = "./" # save location
shape_dict = {} # (layer name:shape) save dictionary

In [ ]:
#To print the value of weights in each layer of model.
for layer in model.layers:
  print(layer.get_weights())
  print("layer finished")

In [ ]:
for layer in model.layers:
    if layer.get_weights() != []:
        shape_dict[layer.name] = np.shape(layer.get_weights()[2]) # No bias, only weights saved (initialized bias=0)
        np.savetxt(loc + layer.name +".csv", layer.get_weights()[2].flatten(), delimiter=",")